In [32]:
from bs4 import BeautifulSoup as bs
from lxml import html
import requests
import pymongo
from splinter import Browser
import time
import pandas as pd
import numpy as np
import string


## initialize browser and login

In [33]:
# initialize browser
executable_path = {"executable_path": "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

### insert profile_id # to search HERE

In [34]:
#define login url (need to add username and password) 
#define site to visit after being logged in
pof_login_url = 'https://login.pof.com'

# update with new profile id here
pof_url = 'https://www.pof.com/viewprofile.aspx?profile_id=166629058'

### input login credentials HERE

In [35]:
browser.visit(pof_login_url) #visit login page

# add credentials
browser.fill("username", "akela_1111") 
browser.fill("password", "plentyoffish")

browser.click_link_by_id("logincontrol_submitbutton") # click submit to enter credentials

In [36]:
browser.visit(pof_url) # browse specified profile

In [37]:
html = browser.html
soup = bs(html, "html.parser")

In [38]:
profile_dict = {}

## username and headline

In [39]:
# use bs and splinter to find username and headline
user = soup.find(id = "username").text
headline = soup.find(id = "headline").text

print(user)
print(headline)

#add to dict
profile_dict["username"] = user
profile_dict["headline"] = headline

Buckwheat1001
I Don't Dance!


## Info in profile table 1


In [40]:
# use bs and splinter to find first table of profile information
# this is done in parts
# part 1: find header names for first profile table

about_head = soup.find_all("div", class_= "profileheadtitle")
headers = []
for info in about_head:
    headers.append(info.text.replace("\n", ""))
headers

['About',
 'City',
 'Details',
 'Ethnicity',
 'Intent',
 'Education',
 'Personality',
 'Profession']

In [41]:
# part 2: find first half of content (left side of table)
about_content_1 = soup.find_all("div", class_="profileheadcontent")
content = []
for info in about_content_1:
    content.append(info.text.replace("\n\n", "").replace("\n", " "))
len(content)

4

In [42]:
# part 3: find 2nd half of content (right side of table)

about_content_2 = soup.find_all("div", class_="profileheadcontent-narrow")
content_2 = []
for info in about_content_2:
    content_2.append(info.text.replace("\n\n", "").replace("\n", " "))
len(content_2)

4

In [43]:
# part 4: combine both content into one list
all_content = np.array([content, content_2])
all_content_ordered = np.ravel(all_content, order = "F")
all_content_ordered

array(['Non-Smoker with Athletic body type', ' Marble falls, Texas ',
       '64 year old Male, 5\' 9" (175cm), Christian - other',
       'Caucasian, Capricorn',
       'Buckwheat1001 is looking for a relationship.', 'Some college',
       'Adventurer', 'Logistics'], dtype='<U50')

In [44]:
# define dictionary to hold data in the first profile table. keys = header/title, value = content

profile_info_1 = dict(zip(headers, all_content_ordered))
profile_info_1

{'About': 'Non-Smoker with Athletic body type',
 'City': ' Marble falls, Texas ',
 'Details': '64 year old Male, 5\' 9" (175cm), Christian - other',
 'Ethnicity': 'Caucasian, Capricorn',
 'Intent': 'Buckwheat1001 is looking for a relationship.',
 'Education': 'Some college',
 'Personality': 'Adventurer',
 'Profession': 'Logistics'}

In [45]:
# add to dict
profile_dict["profile_info_1"] = profile_info_1

## Info in profile table 2

In [46]:
about_head_2 = soup.find_all("span", class_="headline")

headers_2 = []
for i in range(9, 25):
    headers_2.append(about_head_2[i].text.replace("\n",""))
    
headers_2

['I am Seeking a',
 'For',
 ' Needs Test',
 'Chemistry',
 'Do you drink?',
 'Do you want children?',
 'Marital Status',
 'Do you do drugs?',
 'Hair Color',
 'Eye Color',
 'Do you have a car?',
 'Do you have children?',
 'Longest Relationship',
 'How ambitious are you?',
 'Pets',
 'Interests']

In [47]:
about_content_3 = soup.find_all("table")[1]("span", class_="txtGrey")
content_3 = []
for info in about_content_3:
    content_3.append(info.text.replace("\n", ""))
content_3

['Woman',
 'Long Term',
 'Viewhis\xa0relationship needs',
 'View his chemistry results',
 'Socially',
 'Prefer Not To Say',
 'Divorced',
 'No',
 'Brown',
 'Blue',
 'Yes',
 'No',
 'Over 10 years',
 'Ambitious',
 'Cat']

In [48]:
profile_info_2 = dict(zip(headers_2, content_3))
profile_info_2

{'I am Seeking a': 'Woman',
 'For': 'Long Term',
 ' Needs Test': 'Viewhis\xa0relationship needs',
 'Chemistry': 'View his chemistry results',
 'Do you drink?': 'Socially',
 'Do you want children?': 'Prefer Not To Say',
 'Marital Status': 'Divorced',
 'Do you do drugs?': 'No',
 'Hair Color': 'Brown',
 'Eye Color': 'Blue',
 'Do you have a car?': 'Yes',
 'Do you have children?': 'No',
 'Longest Relationship': 'Over 10 years',
 'How ambitious are you?': 'Ambitious',
 'Pets': 'Cat'}

In [49]:
# add to dict
profile_dict["profile_info_2"] = profile_info_2

## Interests

In [50]:
# interests
try:
    interests = soup.find(id="interests").text.split("\n")[1:4]
except: 
    interests = "None"
    print("No interests were entered")

interests

['Boating', 'Water sports', 'PBR']

In [51]:
# add to dict
profile_dict["interests"] = interests

## About Me

In [61]:
about_me = soup.find(id="description").text.replace("\n", "").replace(".", " ")





about_me = about_me.translate(str.maketrans('', '', string.punctuation))

about_me_split = about_me.split(" ")

about_me_split

['First',
 'of',
 'all',
 'most',
 'people',
 'guess',
 'my',
 'age',
 '1015',
 'years',
 'younger',
 'The',
 'real',
 'deal',
 'is',
 'much',
 'better',
 'than',
 'pics',
 'I',
 'stay',
 'active',
 '',
 'Usually',
 'have',
 'a',
 'project',
 'or',
 '3',
 'going',
 'on',
 'around',
 'my',
 'house',
 'at',
 'Lake',
 'Travis',
 '',
 'I',
 'enjoy',
 'boating',
 'water',
 'sports',
 '',
 'hunting',
 '',
 'I',
 'havent',
 'listed',
 'fishing',
 'but',
 'I',
 'do',
 'live',
 'on',
 'Lake',
 'Travis',
 'have',
 'a',
 'dock',
 'and',
 'fishing',
 'gear',
 'so',
 'if',
 'you',
 'like',
 'to',
 'fish',
 'I',
 'can',
 'accomodate',
 '',
 'Im',
 'a',
 'fan',
 'of',
 'the',
 'PBR',
 'and',
 'attend',
 'their',
 'World',
 'Finals',
 'each',
 'year',
 'in',
 'Vegas',
 'Ive',
 'never',
 'used',
 'tobacco',
 'of',
 'any',
 'kind',
 'and',
 'profanity',
 'is',
 'not',
 'part',
 'of',
 'my',
 'vocabulary',
 'I',
 'tell',
 'people',
 'that',
 'I',
 'like',
 'both',
 'kinds',
 'of',
 'music',
 'Country',
 

In [53]:
profile_dict["about_me_text"] = about_me
profile_dict["about_me_split"] = about_me_split

In [54]:
#pd.read_html(pof_url) #403 error - Forbidden???

## dictionary with data of profile_1

In [55]:
profile_dict

{'username': 'Buckwheat1001',
 'headline': "I Don't Dance!",
 'profile_info_1': {'About': 'Non-Smoker with Athletic body type',
  'City': ' Marble falls, Texas ',
  'Details': '64 year old Male, 5\' 9" (175cm), Christian - other',
  'Ethnicity': 'Caucasian, Capricorn',
  'Intent': 'Buckwheat1001 is looking for a relationship.',
  'Education': 'Some college',
  'Personality': 'Adventurer',
  'Profession': 'Logistics'},
 'profile_info_2': {'I am Seeking a': 'Woman',
  'For': 'Long Term',
  ' Needs Test': 'Viewhis\xa0relationship needs',
  'Chemistry': 'View his chemistry results',
  'Do you drink?': 'Socially',
  'Do you want children?': 'Prefer Not To Say',
  'Marital Status': 'Divorced',
  'Do you do drugs?': 'No',
  'Hair Color': 'Brown',
  'Eye Color': 'Blue',
  'Do you have a car?': 'Yes',
  'Do you have children?': 'No',
  'Longest Relationship': 'Over 10 years',
  'How ambitious are you?': 'Ambitious',
  'Pets': 'Cat'},
 'interests': ['Boating', 'Water sports', 'PBR'],
 'about_me_